In [5]:
# 載入所需套件
import requests, numpy as np, pandas as pd, re
from IPython.display import clear_output
import os, datetime

In [ ]:
# Game Information
df_indie = pd.read_excel('Game_info.xlsx')[['Indie_id','Indie_name','Indie_release']]
def string2date(string):
    month_dict = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4
                 ,'May':5, 'Jun':6, 'Jul':7, 'Aug':8
                 ,'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
    'Oct 25, 2022'
    month = month_dict[string.split(' ')[0]]
    day = int(string.split(' ')[1][:-1])
    year = int(string.split(' ')[2])
    return datetime.date(year, month, day)
df_indie['Indie_release'] = df_indie['Indie_release'].apply(lambda x : string2date(x))

In [ ]:
# Data Preprocessing
for index, row in df_indie.iterrows():
    if index > 20:
        break
    # Read Reviews
    game_id = row['Indie_id']
    if game_id in [1332010, 1794680, 648800]:
        continue
    df = pd.read_csv(f'data\\{game_id}.csv')
    df = df[df.review.str.contains('張貼於：2022 年')]

    # retrieve date
    pattern_date = r'張貼於：([0-9]+) 年 ([0-9]+) 月 ([0-9]+) 日'
    df['year'] = df['review'].apply(lambda x : re.search(pattern_date, x).group(1)).astype(int)
    df['month'] = df['review'].apply(lambda x : re.search(pattern_date, x).group(2)).astype(int)
    df['day'] = df['review'].apply(lambda x : re.search(pattern_date, x).group(3)).astype(int)
    df['date'] = df.apply(lambda x : datetime.date(x['year'], x['month'], x['day']), axis=1)

    # retrieve helpful vote
    def helpful_extraction(text):
        pattern_help = r'([0-9]+) 個人認為這篇評論值得參考'
        try:
            return re.search(pattern_help, text).group(1)
        except:
            return 0
    df['helpful_votes'] = df['helpful'].apply(lambda x : helpful_extraction(x)).astype(int)

    # retrieve review
    def review_sub(text):
        text = re.sub(r'張貼於：([0-9]+) 年 ([0-9]+) 月 ([0-9]+) 日', '', text)
        text = re.sub(r'(\D+)評論', '', text)
        text = re.sub(r'\n', '', text)
        text = re.sub(r'\t', '', text)
        text = re.sub(r'產品已退款', '', text)
        text = re.sub(r'免費獲得的產品', '', text)
        return text.strip()
    df['review_clear'] = df['review'].apply(lambda x : review_sub(x)).astype(str)

    # filter early access review
    df = df[df['date'] >= row['Indie_release']]

    # export 
    df.drop(['review', 'helpful', 'year', 'month', 'day'], axis = 1, inplace = True)
    df.to_csv(f'data_cleaned\\{game_id}_cleaned.csv', index = False)

In [ ]:
# Statisitcs
for index, row in df_indie.iterrows():
    if row['Indie_id'] in [1332010, 1794680, 648800]:
        continue
    print(index, row['Indie_id'], row['Indie_name'])
    game_id, game_name = row['Indie_id'], row['Indie_name']
    df = pd.read_csv(f'data_cleaned\\{game_id}_cleaned.csv')
    df['date'] = pd.to_datetime(df['date']).dt.date
    df = df[df['date'] <= (row['Indie_release'] + datetime.timedelta(days = 30))]
    df.to_excel(f'{game_id}.xlsx', index = False)
    print('reviews:', df.shape[0])
    print('length:',df['review_clear'].apply(lambda x : len(str(x))).mean())
    print('helpful votes:', df['helpful_votes'].mean())
    print('---')
    if index >= 19:
        break